In [5]:
using Revise
using PastaQ
using ITensors
using Random
using OptimKit
using Zygote
using Zygote: ChainRulesCore
using BenchmarkTools
using JLD
using MethodAnalysis
using Plots

In [6]:
@time begin
import mVQE
using mVQE.ITensorsExtension: projective_measurement
using mVQE.StateFactory: random_MPS, infinite_temp_MPO
using mVQE.Layers: Rylayer, CXlayer, Rxlayer
using mVQE.Circuits: runcircuit, VariationalCircuitRy, VariationalMeasurement
using mVQE: loss, optimize_and_evolve
end

  0.000294 seconds (702 allocations: 64.688 KiB)


In [7]:
N_state = 5
ancilla_frequency = 1
N = N_state * (ancilla_frequency + 1) - ancilla_frequency
ancillas_indices = [i for i in 1:N if mod1(i, ancilla_frequency+1)!=1]
state_indices = [i for i in 1:N if mod1(i, ancilla_frequency+1)==1];

In [4]:
function hamiltonian(state_indices, h)
    os = OpSum()
    for (i, s) in enumerate(state_indices)
        os += -1, "Z", s, "Z", state_indices[mod1(i+1, length(state_indices))]
        os += -h, "X", s
    end
    
    return os
end

hamiltonian (generic function with 1 method)

In [49]:
function ghz_parent(state_indices, hilbert)
    N = length(hilbert)
    state = zeros(Int, N)
    ψ_0 = productstate(hilbert, state)
    
    state[state_indices] .= 1
    ψ_1 = productstate(hilbert, state)
    ghz = (ψ_0 + ψ_1) / sqrt(2)
    return -outer(ghz, ghz')
end

function mul_operator(state_indices, hilbert; op="X")
    tensors = Vector{ITensor}(undef, length(hilbert))
    for (i, si) in enumerate(hilbert)
        if i in state_indices
            tensors[i] = gate(op, si)
        else
            tensors[i] = gate("Rx", si, θ = 0.) # Identity
        end
    end
    return MPO(tensors)
end

function ghz_parent2(state_indices, hilbert)
    os = OpSum()
    for (i, s) in enumerate(state_indices)
        os += -1, "Z", s, "Z", state_indices[mod1(i+1, length(state_indices))]
    end
    op1 = MPO(os, hilbert) / length(state_indices)
    op2 = mul_operator(state_indices, hilbert; op="X")
    return (op1 - op2)/2
end

ghz_parent2 (generic function with 1 method)

In [50]:
h = -1.  # transverse magnetic field

# Hilbert space
hilbert = qubits(N)

# build MPO "cost function"
H = MPO(hamiltonian(state_indices, h), hilbert);
H = ghz_parent2(state_indices, hilbert)

hilbert_s = hilbert[state_indices]
H_s = MPO(hamiltonian(1:N_state, h), hilbert_s);
H_s = ghz_parent2(1:N_state, hilbert_s)

MPO
[1] ((dim=2|id=231|"Qubit,Site,n=1")', (dim=2|id=231|"Qubit,Site,n=1"), (dim=3|id=749|"Link,l=1"))
[2] ((dim=2|id=576|"Qubit,Site,n=3")', (dim=2|id=576|"Qubit,Site,n=3"), (dim=5|id=586|"Link,l=2"), (dim=3|id=749|"Link,l=1"))
[3] ((dim=2|id=635|"Qubit,Site,n=5")', (dim=2|id=635|"Qubit,Site,n=5"), (dim=5|id=833|"Link,l=3"), (dim=5|id=586|"Link,l=2"))
[4] ((dim=2|id=237|"Qubit,Site,n=7")', (dim=2|id=237|"Qubit,Site,n=7"), (dim=3|id=556|"Link,l=4"), (dim=5|id=833|"Link,l=3"))
[5] ((dim=2|id=29|"Qubit,Site,n=9")', (dim=2|id=29|"Qubit,Site,n=9"), (dim=3|id=556|"Link,l=4"))


In [45]:
#Find ground state with DMRG

nsweeps = 200
maxdim = 10
cutoff_ = 1e-10

start_mps = randomMPS(hilbert, linkdims=10)
Edmrg, Φ = dmrg(H, start_mps; outputlevel=0, nsweeps, maxdim);
println("Ground state energy from DMRG: $Edmrg")
maxlinkdim(Φ)

Ground state energy from DMRG: -2.9999999999999996


8

In [51]:
start_mps = randomMPS(hilbert_s, linkdims=100)
Edmrg, Φ_state = dmrg(H_s, start_mps; outputlevel=0, nsweeps, maxdim=100);
println("Ground state energy from DMRG: $Edmrg")
maxlinkdim(Φ_state)

Ground state energy from DMRG: -1.0000000000000002


2

In [55]:
ψ_0 = productstate(hilbert_s, fill(0, N_state))
ψ_1 = productstate(hilbert_s, fill(1, N_state))

ghz = (ψ_0 + ψ_1)/sqrt(2)
inner(ghz', H_s, ghz)

-1.0

In [56]:
#ghz = (ψ_0 + ψ_1)/2
norm(ghz)

0.9999999999999999

In [57]:
inner(ψ_1, H_s, ψ_1)

-0.5

In [627]:
norm(ghz)

0.9999999999999999

In [628]:
ψ_0 = productstate(hilbert_s, fill(0, N_state))
ψ_1 = productstate(hilbert_s, fill(0, N_state))
α = 0.1
ghz2 = (sqrt(α) * ψ_0 + sqrt(1-α) * ψ_1)
ghz2 /=norm(ghz2)
inner(ghz2', H_s, ghz2)

-0.4999999999999999

In [629]:
inner(ψ_0, H_s, ψ_0), inner(ψ_1, H_s, ψ_1)

(-0.4999999999999999, -0.4999999999999999)

In [630]:

p0 = inner(ψ_0, Φ_state)
p1 = inner(ψ_1, Φ_state)

p0 ^2 + p1^2

0.9999999999999967

In [631]:
p0 ^2, p1 ^2

(0.49999999999999833, 0.49999999999999833)

In [8]:
# contract(Φ.data).tensor[:, 1, :]

In [633]:
Dict("a"=>5)

Dict{String, Int64} with 1 entry:
  "a" => 5

In [636]:
Dict{String, Any}()

Dict{String, Any}()

## Random ansatz

In [8]:
depth = 6
model = VariationalCircuitRy(N, depth)
ψ = productstate(hilbert, fill(0, N))
θs = initialize_circuit(model);

In [9]:
ψout = runcircuit(ψ, model, θs, maxdim=2)
maxlinkdim(ψout)

2

## Variational ansatz

In [10]:
noise_ = (1 => ("depolarizing", (p = 1e-4,)), 
         2 => ("depolarizing", (p = 0.0,)))

Random.seed!(1234)

# run VQE using BFGS optimization
optimizer = LBFGS(; maxiter=50, verbosity=1)

LBFGS{Float64, HagerZhangLineSearch{Rational{Int64}}}(8, 50, 1.0e-8, true, HagerZhangLineSearch{Rational{Int64}}(1//10, 9//10, 1//1000000, 1//2, 2//3, 5//1, 9223372036854775807, -1), 1)

# With errors

In [9]:
optimizer = LBFGS(; maxiter=10, verbosity=1)

LBFGS{Float64, HagerZhangLineSearch{Rational{Int64}}}(8, 10, 1.0e-8, true, HagerZhangLineSearch{Rational{Int64}}(1//10, 9//10, 1//1000000, 1//2, 2//3, 5//1, 9223372036854775807, -1), 1)

In [15]:
#ρs = infinite_temp_MPO(hilbert)

ψ = productstate(hilbert, fill(0, N))
ρs = outer(ψ, ψ')

k = 2
depth = 5
model = VariationalCircuitRy(N, depth)
@time loss_value, θs_error, ρ, niter = optimize_and_evolve(k, ancillas_indices, ρs, H, model, depth; optimizer=optimizer, verbose=true, maxdims, noise=noise_);
ρ, = projective_measurement(ρ; indices=ancillas_indices, reset=1);

┌ Warning: LBFGS: not converged to requested tol: f = -2.824928724859, ‖∇f‖ = 2.3391e-02
└ @ OptimKit /Users/alcalde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:141


iter: 1
Loss: -2.824928724858896

iter: 2
Loss: -2.822275682606981



┌ Warning: LBFGS: not converged to requested tol: f = -2.822275682607, ‖∇f‖ = 3.8798e-02
└ @ OptimKit /Users/alcalde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:141


210.672968 seconds (445.25 M allocations: 23.668 GiB, 5.94% gc time, 98.89% compilation time)


# Optimize through the entire loop

In [ ]:
optimizer = LBFGS(; maxiter=100, verbosity=2)

ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=ancillas_indices, reset=1)

ψ = productstate(hilbert, fill(0, N))
ρs = outer(ψ, prime(ψ))

depth = 5
model = VariationalCircuitRy(N, depth)
reset = 1
k = 4
model = VariationalMeasurement(model, k, ancillas_indices, reset)
@time loss_value, θs_error2, ρ2, niter = optimize_and_evolve(ρs, H, model; optimizer, noise=noise_, maxdim=50);
ρ2, = projective_measurement(ρ2; indices=ancillas_indices, reset=1);

In [38]:
θs = initialize_circuit(model);

In [39]:
loss(ρs, H, model, θs)

0.6758652705238788

In [202]:
inner(Φ, ρ2, Φ')

0.0002890192316163327 + 0.0im

# VQE

In [28]:
ψ = productstate(hilbert_s, fill(0, N_state))
ρs_s = outer(ψ, ψ')

model = VariationalCircuitRy(N_state, 8)


optimizer = LBFGS(; gradtol=1e-8, maxiter=150, verbosity=1)
@time loss_value, θs, ρ_s, niter = optimize_and_evolve(ρs_s, H_s, model, depth; optimizer=optimizer, verbose=true, maxdims, noise);
println(loss_value/Edmrg)

 50.640185 seconds (48.41 M allocations: 11.965 GiB, 5.95% gc time)
0.974617613731652


┌ Warning: LBFGS: not converged to requested tol: f = -6.307857700209, ‖∇f‖ = 1.1731e-02
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:141


In [204]:
inner(ρ_s, H_s)

-6.251523086913398 + 0.0im

In [205]:
inner(Φ_state, ρ_s, Φ_state')

0.5202533766943219 + 0.0im

# del

In [642]:
using PastaQ
using ITensors
using Random
using OptimKit
using Zygote
using Zygote: ChainRulesCore
using BenchmarkTools
using LinearAlgebra
using JLD2
using Flux

import mVQE
using mVQE.Hamiltonians: hamiltonian_tfi, hamiltonian_ghz
using mVQE.ITensorsExtension: projective_measurement
using mVQE: loss, optimize_and_evolve
using mVQE.Circuits: VariationalCircuitRy, VariationalMeasurement, VariationalMeasurementMC, VariationalMeasurementMCFeedback
using mVQE.Misc: get_ancillas_indices, pprint
using mVQE.Optimizers: OptimizerWrapper


instance_number = 2

depths = [4, 6]
depth = depths[mod1(instance_number, length(depths))]

params = Dict{String, Any}("depth" => depth, 
              "maxiter" => 300,
              "maxdim" => 60,
              "noise_p" => 0,
              "N_state" => 6,
              "ancilla_frequency" => 1,
              "k" => 2,
              #"h" => 1.,
              "gradtol" => 1e-3,
              "num_threads" => 1,
              "nthreads" => 18,
              "samples" => 36)


# Create a directory to save the results
save_folder = "save_folder/"
mkpath(save_folder)


if instance_number <= length(depths)
    params["optimizer"] = LBFGS(; gradtol=params["gradtol"], maxiter=params["maxiter"], verbosity=2)
    save_file = "$save_folder/depth=$(depth)_LBFGS.jld2"
elseif instance_number <= 2*length(depths)
    params["optimizer"] = OptimizerWrapper(ADAM(0.1); gradtol=params["gradtol"], maxiter=params["maxiter"], verbosity=2)
    save_file = "$save_folder/depth=$(depth)_ADAM.jld2"
elseif instance_number <= 3*length(depths)
    params["optimizer"] = OptimizerWrapper(RADAM(0.1); gradtol=params["gradtol"], maxiter=params["maxiter"], verbosity=2)
    save_file = "$save_folder/depth=$(depth)_RADAM.jld2"

elseif instance_number <= 4*length(depths)
    params["optimizer"] = OptimizerWrapper(RMSProp(0.1); gradtol=params["gradtol"], maxiter=params["maxiter"], verbosity=2)
    save_file = "$save_folder/depth=$(depth)_RMSProp.jld2"
end


pprint(params)
println()



println(save_file)
println()

# Set number of threads
BLAS.set_num_threads(params["num_threads"])

# Set noise
noise = (1 => ("depolarizing", (p = params["noise_p"],)), 
         2 => ("depolarizing", (p = 0.0,)))

# define ancillas and state indices
state_indices, ancillas_indices, N = get_ancillas_indices(params["N_state"], params["ancilla_frequency"])

# build MPO "cost function"
hilbert = qubits(N)

#H = MPO(hamiltonian_tfi(state_indices, params["h"]), hilbert)
H = hamiltonian_ghz(state_indices, hilbert)

# VQE
Random.seed!(1)

# Initialize state
ψ = productstate(hilbert, fill(0, N))
ρs = outer(ψ, ψ')

function callback(; kwargs...)
    kwargs = Dict(string(key) => v for (key, v) in kwargs)
    kwargs["params"] = params
    save(save_file, kwargs)
end

#= Load previous results
if isfile(save_file)# && false
    previous = load(save_file)

    #@assert previous["params"] == params, "Previous parameterr: $(previous["params"]) != current parameters: $(params)"
    extra_kwargs = (k_in=previous["i"], θs=previous["θs"], misc=previous["optim_misc"])
    θs = previous["θs"]
    misc = previous["optim_misc"]
    println("restarting at k_in=$(previous["i"])/$(params["k"])")
else
    extra_kwargs = Tuple()
end=#

# Run sequential optimization
#@time loss_value, θs, ρ, misc = optimize_and_evolve(params["k"], ancillas_indices, ρs, H, model;
#                                                    optimizer=optimizer, verbose=true, maxdim=params["maxdim"], cutoff=params["cutoff"], noise, callback, extra_kwargs...)

# Define circuit blueprint
vmodels = [VariationalCircuitRy(N, depth) for _ in 1:params["k"]]
#model = VariationalMeasurementMCFeedback(vmodels, [Flux.Dense for i in 2:params["k"]], ancillas_indices)
model = VariationalMeasurementMC(vmodels, ancillas_indices, params["samples"])

# Run optimization
@time loss_value, trained_model, ρ, misc = optimize_and_evolve(ρs, H, model; samples=params["samples"],
                                           optimizer=params["optimizer"], verbose=true, noise, maxdim=params["maxdim"],
                                           parallel=true, nthreads=params["nthreads"])

ρ, = projective_measurement(ρ; indices=ancillas_indices, reset=1)

callback(; loss_value=loss_value, model=trained_model, ρ=ρ, misc=misc, misc=misc)

optimizer => LBFGS{Float64, HagerZhangLineSearch{Rational{Int64}}}(8, 300, 0.001, true, HagerZhangLineSearch{Rational{Int64}}(1//10, 9//10, 1//1000000, 1//2, 2//3, 5//1, 9223372036854775807, 0), 2)
maxiter => 300
k => 2
ancilla_frequency => 1
depth => 6
nthreads => 18
noise_p => 0
N_state => 6
samples => 36
gradtol => 0.001
maxdim => 60
num_threads => 1

save_folder//depth=6_LBFGS.jld2



LoadError: AssertionError: reset <= dim(s)

In [640]:
model = VariationalCircuitRy(N, depth)

VariationalCircuitRy(N=11, depth=6)

In [641]:
loss(ρs, H, model)

-0.0003190539717932331